# Variational Autoencoder with Normalizing Flows

In [1]:
# Import packages
import numpy as np
import torch
from torch import nn, optim
from torchvision import datasets, transforms

from tqdm import tqdm
from matplotlib import pyplot as plt
import sys
sys.path.append('/home/zhengwei/Desktop/Zhengwei/Projects/CVAE/normalizing-flows')
import normflows as nf

In [2]:
# Hyperparameters
torch.manual_seed(0)

batch_size = 64
num_samples = 32
n_flows = 40
n_bottleneck = 40
hidden_units_encoder = np.array([28 ** 2, 512, 256, n_bottleneck * 2])
hidden_units_decoder = np.array([n_bottleneck, 256, 512, 28 ** 2])
flow_type = 'RealNVP'
n_epochs = 15

enable_cuda = True
device = torch.device('cuda' if torch.cuda.is_available() and enable_cuda else 'cpu')

In [3]:
# Get dataloader
class BinaryTransform():
    def __init__(self, thresh=0.5):
        self.thresh = thresh

    def __call__(self, x):
        return (x > self.thresh).type(x.type())
transform=transforms.Compose([transforms.ToTensor(), BinaryTransform()])
mnist_train = datasets.MNIST('../datasets', train=True, download=True, transform=transform)
mnist_test = datasets.MNIST('../datasets', train=False, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size, shuffle=True)

In [4]:
# Set up model
prior = torch.distributions.MultivariateNormal(torch.zeros(n_bottleneck, device=device),
                                               torch.eye(n_bottleneck, device=device))
encoder_nn = nf.nets.MLP(hidden_units_encoder)
encoder = nf.distributions.NNDiagGaussian(encoder_nn)
decoder_nn = nf.nets.MLP(hidden_units_decoder)
decoder = nf.distributions.NNBernoulliDecoder(decoder_nn)

if flow_type == 'Planar':
    flows = [nf.flows.Planar((n_bottleneck,)) for k in range(n_flows)]
elif flow_type == 'Radial':
    flows = [nf.flows.Radial((n_bottleneck,)) for k in range(n_flows)]
elif flow_type == 'RealNVP':
    b = torch.tensor(n_bottleneck // 2 * [0, 1] + n_bottleneck % 2 * [0])
    flows = []
    for i in range(n_flows):
        s = nf.nets.MLP([n_bottleneck, n_bottleneck // 2, n_bottleneck], init_zeros=True)
        t = nf.nets.MLP([n_bottleneck, n_bottleneck // 2, n_bottleneck], init_zeros=True)
        if i % 2 == 0:
            flows += [nf.flows.MaskedAffineFlow(b, t, s)]
        else:
            flows += [nf.flows.MaskedAffineFlow(1 - b, t, s), nf.flows.BatchNorm()]
else:
    raise NotImplementedError

flows = flows[:-1]
nfm = nf.NormalizingFlowVAE(prior, encoder, flows, decoder)
nfm.to(device)

NormalizingFlowVAE(
  (decoder): NNBernoulliDecoder(
    (net): MLP(
      (net): Sequential(
        (0): Linear(in_features=40, out_features=256, bias=True)
        (1): LeakyReLU(negative_slope=0.0)
        (2): Linear(in_features=256, out_features=512, bias=True)
        (3): LeakyReLU(negative_slope=0.0)
        (4): Linear(in_features=512, out_features=784, bias=True)
      )
    )
  )
  (flows): ModuleList(
    (0): MaskedAffineFlow(
      (s): MLP(
        (net): Sequential(
          (0): Linear(in_features=40, out_features=20, bias=True)
          (1): LeakyReLU(negative_slope=0.0)
          (2): Linear(in_features=20, out_features=40, bias=True)
        )
      )
      (t): MLP(
        (net): Sequential(
          (0): Linear(in_features=40, out_features=20, bias=True)
          (1): LeakyReLU(negative_slope=0.0)
          (2): Linear(in_features=20, out_features=40, bias=True)
        )
      )
    )
    (1): MaskedAffineFlow(
      (s): MLP(
        (net): Sequential(
   

In [5]:
# Train model
log_intv = 100
optimizer = optim.Adam(nfm.parameters(), lr=1e-4, weight_decay=1e-4)
for epoch in range(n_epochs):
    progressbar = tqdm(enumerate(train_loader), total=len(train_loader))
    for batch_n, (x, n) in progressbar:
        x = x.to(device)
        optimizer.zero_grad()
        z, log_q, log_p = nfm(x.view(x.size(0) * x.size(1), 28 ** 2), num_samples)
        
        recon_x = nfm.decoder(z)
        mean_log_q = torch.mean(log_q)
        mean_log_p = torch.mean(log_p)

        print("mean_log_p,shape{}, {}".format(mean_log_p.shape, mean_log_p))
        print("mean_log_q.shape{}, {}".format(mean_log_q.shape, mean_log_q))
        kl_loss = mean_log_q - mean_log_p
        try:
            x_expand = x.view(64, 1, -1).expand_as(recon_x)
        except:
            x_expand = x.view(32, 1, -1).expand_as(recon_x)
    
        recon_loss = torch.nn.functional.binary_cross_entropy(recon_x, x_expand, reduction='mean')
        
        loss = kl_loss + recon_loss
        loss.backward()
        optimizer.step()
        progressbar.update()
    progressbar.close()
    print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_n * len(x), len(train_loader.dataset),
                       100. * batch_n / len(train_loader),
                       loss.item()))

  0%|          | 1/938 [00:00<01:51,  8.42it/s]

mean_log_p,shapetorch.Size([]), -600.88232421875
mean_log_q.shapetorch.Size([]), -56.74915313720703
mean_log_p,shapetorch.Size([]), -599.038818359375
mean_log_q.shapetorch.Size([]), -56.733726501464844
mean_log_p,shapetorch.Size([]), -597.2940673828125
mean_log_q.shapetorch.Size([]), -56.744258880615234


  1%|          | 8/938 [00:00<00:47, 19.47it/s]

mean_log_p,shapetorch.Size([]), -595.3580322265625
mean_log_q.shapetorch.Size([]), -56.747657775878906
mean_log_p,shapetorch.Size([]), -593.6387939453125
mean_log_q.shapetorch.Size([]), -56.754188537597656
mean_log_p,shapetorch.Size([]), -591.471923828125
mean_log_q.shapetorch.Size([]), -56.7750244140625


  1%|▏         | 12/938 [00:00<00:43, 21.40it/s]

...

mean_log_p,shapetorch.Size([]), -153.9075927734375
mean_log_q.shapetorch.Size([]), -38.93017578125
mean_log_p,shapetorch.Size([]), -154.54263305664062
mean_log_q.shapetorch.Size([]), -37.80706787109375


 55%|█████▌    | 520/938 [00:36<00:34, 12.08it/s]

mean_log_p,shapetorch.Size([]), -151.79348754882812
mean_log_q.shapetorch.Size([]), -38.06523895263672


 51%|█████     | 475/938 [00:36<00:35, 13.08it/s]


KeyboardInterrupt: 

In [1]:
x_out = nfm.decoder(torch.randn((1, n_bottleneck), device=device))
x_np = x_out.view((28, 28)).to('cpu').detach().numpy()
plt.imshow(x_np)
plt.show()

NameError: name 'nfm' is not defined

In [9]:
import torch
q0 = torch.distributions.MultivariateNormal(torch.zeros(64), torch.eye(64))
z = torch.randn(64, 32, 64)

# if is independent, log(q) = logq_1 + logq_2 + ... + logq_n
a= q0.log_prob(z)
print(z.shape)
print(a.shape)

print(z.size()[2:])

z = z.view(-1, *z.size()[2:])

print(z.shape)

print(z.size()[2])



torch.Size([64, 32, 64])
torch.Size([64, 32])
torch.Size([64])
torch.Size([2048, 64])


IndexError: tuple index out of range